In [1]:
from stockfish import Stockfish
import chess.pgn
import io

In [2]:
# stockfish init
stockfish = Stockfish(
    '/home/vini/Personal/test_project/stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
    parameters={"Threads": 2, 'Min Split Depth': 20, 'Ponder':True}
)
stockfish.set_elo_rating(2600)
stockfish.set_skill_level(30)

# read game

In [3]:
file_name = "../raw_data/Fics_data_pc_data.pgn"
pgn = open(file_name, encoding='UTF-8')

In [4]:
def write_file():
    game_headers, game_moves = str(game).split('\n\n')
    game_moves = str(game_moves).split('(')[1].split(')')[0]
    # print(headers)
    # print(game_moves)
    current_game = game_headers+'\n\n'+game_moves
    # print(current_game)

    # current_game = current_game.decode('utf-8')
    current_game = io.StringIO(current_game)
    current_game = chess.pgn.read_game(current_game)

    new_game = open("fics_evals.pgn", "a", encoding="utf-8")
    exporter = chess.pgn.FileExporter(new_game)
    current_game.accept(exporter)

In [5]:
game_counter = 0
while True:
    try:
        game_counter += 1
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        if len(moves) > 5:
            node = game.add_variation(moves[0])
            # print(node)
            game_counter = 0
            for move in moves:
                game_counter += 1
                stockfish.set_fen_position(board.fen())
                eval_ = stockfish.get_evaluation()

                if game_counter == 1:
                    node.comment = '[%eval '+str(eval_['value'])+'] [%clk 0:10:00]'
                else:
                    node = node.add_variation(move)
                    node.comment = '[%eval '+str(eval_['value'])+'] [%clk 0:10:00]'
                board.push(move)
            write_file()
    except AttributeError:
        break
print(game_counter)

19
